# Нахождение собственных векторов (метод степенной итерации с дефляцией)
---
Функция `find_eigenvectors(C, eigenvalues)` извлекает из симметричной матрицы $C\in\mathbb{R}^{n\times n}$ собственные векторы, используя уже найденные собственные значения для управления числом итераций.

---

### Описание алгоритма

1. **Копирование данных**
   Копируем $C$ в двумерный список $A$:
   $$
     A_{ij} = C_{ij},\quad i,j=1\ldots n.
   $$

2. **Параметры итераций**
   Устанавливаем допуск и максимальное число итераций:
   $$
     \mathrm{tol} = 10^{-6},\quad \mathrm{max\_iter} = 1000.
   $$

3. **Цикл по списку `eigenvalues`**
   Для каждого значения $\lambda$ из `eigenvalues` (или просто $n$ раз):

   1. **Инициализация вектора**
      Генерируем случайный вектор $b^{(0)}\in\mathbb{R}^n$ и нормируем:
      $$
        b^{(0)} \leftarrow \frac{b^{(0)}}{\|b^{(0)}\|}.
      $$

   2. **Power iteration**
      Для $k = 0,1,\dots$ до сходимости или пока $k<\mathrm{max\_iter}$ выполняем:
      $$
        y = A\,b^{(k)},\quad
        b^{(k+1)} = \frac{y}{\|y\|}.
      $$
      Вычисляем новое приближение собственного значения через Rayleigh-отношение:
      $$
        \lambda^{(k+1)} = (b^{(k+1)})^\top A\,b^{(k+1)}.
      $$
      Критерий остановки:
      $$
        \bigl|\lambda^{(k+1)} - \lambda^{(k)}\bigr| < \mathrm{tol}.
      $$

   3. **Сохранение вектора**
      После сходимости строим столбец
      $$
        v = \begin{pmatrix} b^{(k+1)}_1 \\ \vdots \\ b^{(k+1)}_n \end{pmatrix}
      $$
      и добавляем его в список `eigenvectors`.

   4. **Дефляция**
      Убираем найденную компоненту из $A$:
      $$
        A \leftarrow A - \lambda^{(k+1)}\,b^{(k+1)}\,(b^{(k+1)})^\top.
      $$

4. **Возврат результата**
   Возвращаем список `eigenvectors`.

---

### Особенности реализации

- Метод сходится к собственному вектору, соответствующему наибольшему по модулю собственному значению оставшейся матрицы.
- Дефляция обеспечивает извлечение следующих по величине собственных векторов.
- Критерий по допуску $\mathrm{tol}$ гарантирует требуемую точность.
- Ограничение в $\mathrm{max\_iter}=1000$ предотвращает зацикливание на плохо обусловленных матрицах.


---

In [1]:
from src.Matrix import Matrix
from typing import List
import random
import math

In [2]:
def find_eigenvectors(C: Matrix, eigenvalues: List[float]) -> List[Matrix]:
    """
    Вход:
      C: матрица ковариаций (n×n)
      eigenvalues: список собственных значений
    Выход:
      список собственных векторов (каждый — Matrix-столбец)
    """
    n = C.rows
    # Копируем данные в обычный список для дефляции
    A = [[C.data[i][j] for j in range(n)] for i in range(n)]
    eigenvectors: List[Matrix] = []
    tol = 1e-6  # допуск для остановки power iteration

    for _ in eigenvalues:
        # Случайный стартовый вектор
        b = [random.random() for _ in range(n)]
        norm_b = math.sqrt(sum(x*x for x in b)) or 1.0
        b = [x / norm_b for x in b]

        lambda_old = 0.0
        # Power iteration
        for _ in range(1000):
            # умножаем A на b
            Ab = [sum(A[i][j] * b[j] for j in range(n)) for i in range(n)]
            norm_ab = math.sqrt(sum(x*x for x in Ab)) or 1.0
            b = [x / norm_ab for x in Ab]
            lambda_new = sum(b[i] * Ab[i] for i in range(n))
            if abs(lambda_new - lambda_old) < tol:
                break
            lambda_old = lambda_new

        # cохраняем вектор-столбец в формате Matrix
        vec = Matrix([[b[i]] for i in range(n)])
        eigenvectors.append(vec)

        # дефляция: вычитаем найденное собственное значение из матрицы
        for i in range(n):
            for j in range(n):
                A[i][j] -= lambda_old * b[i] * b[j]

    return eigenvectors


---